In [1]:
import os
import cv2
import open3d as o3d
import numpy as np

from PIL import Image
from tqdm import tqdm

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
PATH = '/home/weders/scratch/scratch/scannetter/arkit/raw/Validation'
SCENE = '42445991'

# PATH = '/home/weders/scratch/scratch/scannetter'
# SCENE = 'scene0164_02'

In [3]:
scene_path = os.path.join(PATH, SCENE)
image_path = os.path.join(scene_path, 'color')
depth_path = os.path.join(scene_path, 'depth')
intrinsics_path = os.path.join(scene_path, 'intrinsic')
pose_path = os.path.join(scene_path, 'pose')

In [13]:
tsdf = o3d.pipelines.integration.ScalableTSDFVolume(sdf_trunc=0.06, 
                                                    voxel_length=0.02, 
                                                    color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8)

intrinsics_loaded = False

files = sorted(os.listdir(depth_path), key=lambda x: int(x.split('.')[0]))
pcds = None
resize_image = False

for idx, file in  tqdm(enumerate(files), total=len(files)):
    
    if idx % 5 != 0:
        continue
    
    if not os.path.exists(os.path.join(image_path, file.replace('.png', '.jpg'))):
        print(file, 'not found')
        continue
        
    image = np.asarray(Image.open(os.path.join(image_path, file.replace('.png', '.jpg')))).astype(np.uint8)
    depth = np.asarray(Image.open(os.path.join(depth_path, file))).astype(np.float32) / 1000.    
    
    if resize_image:
        h, w = depth.shape
        image = cv2.resize(image, (w, h))
    else:
        h, w, _ = image.shape
        depth = cv2.resize(depth, (w, h))
    
    if not intrinsics_loaded:
        intrinsics = np.loadtxt(intrinsics_path + '/intrinsic_depth.txt')
        intrinsics = o3d.camera.PinholeCameraIntrinsic(width=w, height=h, fx=intrinsics[0, 0], fy=intrinsics[1, 1], cx=intrinsics[0, 2], cy=intrinsics[1, 2])
        intrinsics_loaded = False
    
    pose_file = os.path.join(pose_path, file.replace('.png', '.txt'))
    pose = np.loadtxt(pose_file)
    # pose = np.linalg.inv(pose)
    
    image = o3d.geometry.Image(image)
    depth = o3d.geometry.Image(depth)
    
    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(image, 
                                                              depth,
                                                              depth_scale=1.0,
                                                              depth_trunc=3.,
                                                              convert_rgb_to_intensity=False)
    
    # if pcds is None:
        
    #    pcds = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsics, np.linalg.inv(pose))
    
    # else:
    #    pcds = pcds + o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsics, np.linalg.inv(pose))
    #    # pcds = pcds.voxel_down_sample(0.01)
    
    tsdf.integrate(rgbd, intrinsics, np.linalg.inv(pose))
    

    

mesh = tsdf.extract_triangle_mesh()
o3d.io.write_triangle_mesh(f'{SCENE}.ply', mesh)

100%|██████████████████████████████████████████████████████████████████████| 2517/2517 [00:15<00:00, 162.96it/s]


True